In [1]:
import pandas as pd
import nltk
import re

In [2]:
d=pd.read_csv("train.csv")
d.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
d=d.drop(['keyword','location','id'],axis=1)
d.tail()

,text,target
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1
7612,The Latest: More Homes Razed by Northern Calif...,1


In [4]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps=PorterStemmer()
c=[]
for i in range(len(d)):
    rp=re.sub('[^a-zA-Z]',' ',d['text'][i])
    rp=rp.lower()
    rp=rp.split()
    rp=[ps.stem(word) for word in rp if not word in set (stopwords.words('english'))]
    rp=" ".join(rp)
    c.append(rp)    

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()
X=cv.fit_transform(c).toarray()
Y=d['target']

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.05,random_state=83)

In [8]:
from sklearn.naive_bayes import MultinomialNB
NB=MultinomialNB(alpha=9.0)
NB.fit(X_train,Y_train)

MultinomialNB(alpha=9.0)

In [9]:
Y_pred=NB.predict(X_test)
Y_pred1=NB.predict(X_train)

In [10]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
print(accuracy_score(Y_train,Y_pred1),accuracy_score(Y_test,Y_pred))
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))

0.8523230088495575 0.8188976377952756
[[185  29]
 [ 40 127]]
              precision    recall  f1-score   support

           0       0.82      0.86      0.84       214
           1       0.81      0.76      0.79       167

    accuracy                           0.82       381
   macro avg       0.82      0.81      0.81       381
weighted avg       0.82      0.82      0.82       381



In [11]:
d1=pd.read_csv("test.csv")

In [12]:
d1=d1.drop(['keyword','location'],axis=1)

In [13]:
d1.tail()

,id,text
3258,10861,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,Storm in RI worse than last hurricane. My city...
3260,10868,Green Line derailment in Chicago http://t.co/U...
3261,10874,MEG issues Hazardous Weather Outlook (HWO) htt...
3262,10875,#CityofCalgary has activated its Municipal Eme...


In [14]:
h=[]
for i in range(len(d1)):
    rp=re.sub('[^a-zA-Z]',' ',d1['text'][i])
    rp=rp.lower()
    rp=rp.split()
    rp=[ps.stem(word) for word in rp if not word in set (stopwords.words('english'))]
    rp=" ".join(rp)
    h.append(rp) 

In [15]:
d1=cv.transform(h).toarray()
pred=NB.predict(d1)

In [16]:
pred

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [17]:
NB.predict_proba(d1)

array([[0.26741336, 0.73258664],
       [0.54214035, 0.45785965],
       [0.06160875, 0.93839125],
       ...,
       [0.17831342, 0.82168658],
       [0.05026078, 0.94973922],
       [0.31302732, 0.68697268]])

In [18]:
df=pd.read_csv("F:/Kaggle/NLP/DataSets/test.csv")
Sub=pd.DataFrame({'id':df['id'],'target':pred})
Sub.to_csv('nlp.csv',index=False)